Below we use 7b param code llama 2 specialized for Python

In [ ]:
# import sentencepiece
# from transformers import LlamaForCausalLM, CodeLlamaTokenizer

# tokenizer = CodeLlamaTokenizer.from_pretrained("codellama/CodeLlama-7b-hf")
# model = LlamaForCausalLM.from_pretrained("codellama/CodeLlama-7b-hf")

In [ ]:
# PROMPT = '''Write a function to find the length of the shortest string that has both str1 and str2 as subsequences.
# def super_seq(X, Y, m, n):
# <FILL_ME>
# return result
# '''
# input_ids = tokenizer(PROMPT, return_tensors="pt")["input_ids"]
# generated_ids = model.generate(input_ids, max_new_tokens=128)

# filling = tokenizer.batch_decode(generated_ids[:, input_ids.shape[1]:], skip_special_tokens = True)[0]
# print(PROMPT.replace("<FILL_ME>", filling))

In [ ]:
def find_def_until_colon(input_string):
    def_index = input_string.find('def')

    if def_index != -1:
        colon_index = input_string.find(':', def_index)

        if colon_index != -1:
            return input_string[def_index:colon_index + 1]

    return None

In [ ]:
def everything_after_def(input_string):
    def_index = input_string.find('def')

    if def_index != -1:
        double_newline_index = input_string.find('\n\n', def_index)
        if double_newline_index != -1:
            result = input_string[def_index:double_newline_index]
            return result
    return None

In [ ]:
def everything_after_def_2(input_string):
    def_index = input_string.find('def')

    if def_index != -1:
        substring_from_def = input_string[def_index:]

        return_index = substring_from_def.rfind('return')
        if return_index != -1:
            newline_index = substring_from_def.find('\n', return_index)
            if newline_index != -1:
                result = substring_from_def[:newline_index]
                return result.strip()
            else:
                result = substring_from_def[:return_index]
                return result.strip()

    return None

In [ ]:
testing = "\ndef is_Monotonic(A):\r\n    return all(A[i] <= A[i + 1] for i in range(len(A) - 1)) or all(A[i] >= A[i + 1] for i in range(len(A) - 1))\r\n"

In [ ]:
(everything_after_def_2(testing) == everything_after_def(testing))
everything_after_def_2(testing)

'def is_Monotonic(A):\r\n    return all(A[i] <= A[i + 1] for i in range(len(A) - 1)) or all(A[i] >= A[i + 1] for i in range(len(A) - 1))'

In [ ]:
# import re

# def extract_args_with_function_name(declaration):
#     pattern = r'def\s+(\w+)\s*\((.*?)\)'

#     match = re.search(pattern, declaration)

#     if match:
#         function_name = match.group(1)
#         arguments = match.group(2).split(', ')
#     else:
#         function_name = ""
#         arguments = []

#     if len(arguments) == 0:
#         return function_name, "that takes no arguments"
#     elif len(arguments) == 1:
#         return function_name, f"that takes {arguments[0]} as an argument"
#     else:
#         args_except_last = ", ".join(arguments[:-1])
#         last_arg = arguments[-1]
#         return function_name, f"that takes {args_except_last} and {last_arg} as arguments"

In [ ]:
def proper_format(prompt_in, code_in):
    # str_1 = " "
    # fun_name, str_2 = extract_args_with_function_name(code_in)
    function_index = prompt_in.find("to")
    if function_index != -1:
        result = prompt_in[function_index + len("to"):].strip()
        return result + " \n" + find_def_until_colon(code_in)
    return ""

In [ ]:
# def getCodeFormat(prompt_in, code_in):
#     prompt = proper_format(prompt_in, code_in)

#     input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]
#     attention_mask = torch.ones_like(input_ids)
#     pad_token_id = tokenizer.eos_token_id

#     generated_ids = model.generate(
#         input_ids,
#         attention_mask=attention_mask,
#         pad_token_id=pad_token_id,
#         max_new_tokens=128,
#         num_return_sequences=1
#     )

#     generated_code = tokenizer.batch_decode(
#         generated_ids[:, input_ids.shape[1]:],
#         skip_special_tokens=True
#     )[0]

#     # Assuming generated_code directly contains the code without placeholders
#     return generated_code


# print(getCodeFormat("Write a python function to find the length of the shortest word.", "def len_log(list1): min=len(list1[0]) for i in list1: if len(i)<min: min=len(i) return min"))

In [ ]:
# def getCodeFormat(prompt_in, code_in):
#     PROMPT = proper_format(prompt_in, code_in)
#     input_ids = tokenizer(PROMPT, return_tensors="pt")["input_ids"]

#     attention_mask = torch.ones_like(input_ids)  # Set attention mask to ones (all tokens are attended to)
#     pad_token_id = tokenizer.eos_token_id  # Get the pad token ID

#     generated_ids = model.generate(input_ids, attention_mask=attention_mask, pad_token_id=pad_token_id, max_new_tokens=128, num_return_sequences=1)
#     filling = tokenizer.batch_decode(generated_ids[:, input_ids.shape[1]:], skip_special_tokens = True)[0]
#     print("hello" + filling)
#     responseText = PROMPT.replace("<FILL_ME>", filling)

#     return filling

# print(getCodeFormat("Write a python function to find the length of the shortest word.", "def len_log(list1): min=len(list1[0]) for i in list1: if len(i)<min: min=len(i) return min"))

In [ ]:
pip install git+https://github.com/huggingface/transformers.git@main accelerate

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "codellama/CodeLlama-7b-Python-hf"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float32,
    device_map="auto",
)

In [ ]:
def generateCode(prompt_input, code_input):
  sequences = pipeline(
      proper_format(prompt_input, code_input),
      do_sample=True,
      top_k=10,
      temperature=1,
      top_p=0.95,
      num_return_sequences=1,
      eos_token_id=tokenizer.eos_token_id,
      pad_token_id=tokenizer.eos_token_id,
      max_length=200,
  )
  return everything_after_def_2(sequences[0]['generated_text'])

# print(getCodeFormat("Write a python function to find the length of the shortest word.", "def len_log(list1): min=len(list1[0]) for i in list1: if len(i)<min: min=len(i) return min"))

In [ ]:
import locale
locale.getpreferredencoding = lambda: 'UTF-8'

In [ ]:
# import jsonlines
# import torch


# max_loop = 20
# i = 0

# with jsonlines.open('mbpp.jsonl') as reader:
# # Loop through each example
#     for obj in reader:
#         prompt = obj['text']
#         code = obj['code']

#         print(find_def_until_colon(code))

#         responseText = getCodeFormat(prompt, code)

#         print(responseText)

#         try:
#           exec(responseText)
#         except Exception as E:
#           print("Code Didn't Run")
#           print("Error" + str(E))
#           continue

#         # Assertion Code:
#         assertion_code = obj['test_list']
#         # Execute and print if the code passes the assertions
#         for assertion in assertion_code:
#             print(assertion)
#             try:
#                 exec(assertion)
#             except Exception as E:
#                 # Continue to the next example and print the error
#                 print("Code Failed Assertions" + str(E))
#                 break

#         # terminate loop for testing
#         i += 1
#         if i == max_loop:
#             break

In [ ]:
pip install jsonlines

In [ ]:
pip install openai

In [ ]:
import time
import jsonlines
import os
# import openai
# from openai import OpenAI
# from dotenv import load_dotenv
import textwrap
import signal
import time
from contextlib import contextmanager

class TimeoutException(Exception): pass

@contextmanager
def time_limit(seconds):
    def signal_handler(signum, frame):
        raise TimeoutException("Timed out!")
    signal.signal(signal.SIGALRM, signal_handler)
    signal.alarm(seconds)
    try:
        yield
    finally:
        signal.alarm(0)


import bisect
import collections
import math
import heapq
import operator
import datetime
import itertools
import cmath
import sys
import re
import array
import copy
from collections import Counter, defaultdict


def checkCorrectness(output, obj):
    try:
      exec(output)
    except:
      print("Code didnt compile")
      success = False
      return False

    assertion_code = obj['test_list']

    success = True
    for assertion in assertion_code:
        try:
            exec(assertion)
            print("Code Passed Assertions")
        except Exception as e:
            # Continue to the next example and print the error
            print(f"Error: {e}")
            print("Code Failed Assertions")
            success = False
    return success

if __name__ == '__main__':

    # #max_loop = 974
    max_loop = 300
    start_line = 151  # Start processing from the 43rd line
    with jsonlines.open('mbpp_label_refined.jsonl') as reader:
        i = sum(1 for _ in reader)

    with jsonlines.open('mbpp.jsonl') as reader:
        # Loop through each example
        for count, obj in enumerate(reader):
            # Start from the last example that was run
            with jsonlines.open('mbpp_label_refined.jsonl') as reader:
                i = sum(1 for _ in reader)
            if count < start_line - 1 or count < i:
                continue

            code_llama_2 = 0
            success_gpt4 = 0

            for _ in range(5):
                print("Running Code Llama 2")
                code_llama_response = generateCode(obj['text'], obj['code'])
                print(code_llama_response)
                if checkCorrectness(code_llama_response, obj):
                    code_llama_2 += 1

                time.sleep(5)

            obj["code_llama_response"] = code_llama_response
            obj["code_llama_success"] = code_llama_2

            with jsonlines.open('mbpp_label_refined.jsonl', mode='a') as writer:
                writer.write(obj)

            if i == max_loop:
                break


            time.sleep(5)